In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
# import LAM library for logbook, data analysis...
from pfs.lam.sacFileHandling import Logbook, constructFilelist
import pfs.lam.imageAnalysis as imeas
from pfs.lam.slitFocusAnalysis import *
from pfs.lam.style import colors

In [3]:
from matplotlib import style
style.use('ggplot')

In [4]:
imgPath = '/home/pfs/shared/Pictures/SM1/SLITALIGN'

In [5]:
smId = 3

In [6]:
# filepath of data model
pmodel = os.environ['LAM_SPS_ANALYSIS_DIR']+"/notebooks/optical/input/slit"
file = 'slit-defParam-Ouverture100-fiber65.csv'

In [7]:
zemaxData = pd.read_csv('%s/%s' % (pmodel, file), delimiter=" ")
zemaxMidFit = imeas.fitparabola(x=zemaxData.Slitdefocus, y=zemaxData.MidFiber, deg=15, focus='max')
zemaxEndFit = imeas.fitparabola(x=zemaxData.Slitdefocus, y=zemaxData.ExtremeFiber, deg=15, focus='max')

In [8]:
experiments = pd.read_sql_query('select * from Experiment where type="slitAlignment" order by experimentId desc',
                                con='sqlite:////data/ait/experimentLog-sac.db', index_col='experimentId')

experiments['exptime'] = [Logbook.getParameter(experimentId, 'exptime') for experimentId in experiments.index]
experiments['fiber'] = [Logbook.getParameter(experimentId, 'fiber', doRaise=False) for experimentId in experiments.index]

In [9]:
experiments

,name,visitStart,visitEnd,type,cmdStr,comments,anomalies,startdate,cmdError,exptime,fiber
experimentId,,,,,,,,,,,
468,SM3_Slit_Alignment,14437,14496,slitAlignment,"spsait slit align exptime=4.0 position=-1.5,2....",halogen mask 125mm,,2021-03-02T10:12:33,,4.0,scibotend
467,SM3_Slit_Alignment,14377,14436,slitAlignment,"spsait slit align exptime=4.0 position=-1.5,2....",halogen mask 125mm,,2021-03-02T09:48:42,,4.0,scibotmid
466,SM3_Slit_Alignment,14317,14376,slitAlignment,"spsait slit align exptime=4.0 position=-1.5,2....",halogen mask 125mm,,2021-03-02T09:24:40,,4.0,scitopmid
465,SM3_Slit_Alignment,14257,14316,slitAlignment,"spsait slit align exptime=4.0 position=-1.5,2....",halogen mask 125mm,,2021-03-02T09:00:35,,4.0,scitopend
464,SM3_Slit_Alignment,14181,14240,slitAlignment,"spsait slit align exptime=1.0 position=-1.5,2....",halogen mask 125mm,,2021-02-19T10:36:38,,1.0,engbotend
...,...,...,...,...,...,...,...,...,...,...,...
40,first_test,1199,1258,slitAlignment,slit throughfocus exptime=3.0 lowBound=-0.5 up...,,,2018-07-16T17:07:37,None,3.0,engtopmid
39,first_test,1196,1198,slitAlignment,slit throughfocus exptime=3.0 lowBound=-0.5 up...,,light was on,2018-07-16T17:05:53,None,3.0,engtopmid
38,first_test,1132,1191,slitAlignment,slit throughfocus exptime=6.0 lowBound=-0.5 up...,,,2018-07-16T16:02:19,None,6.0,engtopmid


## Data required : 
To be able to find the right focus, some data acquisition is required.

you need to use ics_spsaitActor to perform your sequence

example : <b>slit throughfocus exptime=6.0 lowBound=-0.5 upBound=1.5 fiber=engbotend nbPosition=10 duplicate=1 </b>

-0.5 : 1.5 is the range of the slit focus


## Input Parameters : 
The only parameters needed is the experimentIds that match your data acquisition sequence

In [10]:
com = True
doBck =True
corrector = False
experimentIds = [465,466,467,468] 

head = 0
tail = 0
#bck_expId = 260

dfs = []

In [11]:
threshold = 300
radius = 60
doPrint = False
doPlot = False
roi_size = 150

fwhm_radius = radius
fwhm_method = 'gaussian'

In [ ]:
for experimentId in experimentIds:
    visitStart, visitEnd = Logbook.visitRange(experimentId=experimentId)
    filelist = constructFilelist(visitStart=visitStart, visitEnd=visitEnd)
    duplicate = int(Logbook.getParameter(experimentId=experimentId, param='duplicate'))
    filelist = filelist[head*duplicate:len(filelist)-tail*duplicate]
    guessedPos = getSlitPosFromMove(experimentId)

    res = []
    for i in range(len(filelist) // duplicate):
        print(i)
        print(filelist[i])
        hdr, data = stackedImage(filelist=filelist, ind=i, duplicate=duplicate, doBck=doBck)
    #    peak = getPeakData(data, com=com, doBck=False, doPlot=doPlot, doPrint=doPrint)
        if doPlot:
            plt.imshow(data, origin = 'lower', cmap = 'gray', vmin=300, vmax=800)
        calc = iqcalc.IQCalc(None)
        peaks = calc.find_bright_peaks(data, threshold=threshold, radius=radius)
        objlist = calc.evaluate_peaks(peaks, data, fwhm_radius=radius, cb_fn=None, ev_intr=None, fwhm_method='gaussian')
        if doPrint: 
            print(len(objlist))


        objlist = [elem for elem in objlist if (elem['fwhm'] > 15)
                                                and (elem['fwhm_x'] > 15) 
                                                    and (elem['fwhm_y'] > 15)
                                                        and (0 < elem['objx'] < 1940)
                                                            and (0 < elem['objy'] < 1460)
                                                                and (threshold < elem['brightness'] < 50000)]
        for obj in objlist :
            plt.scatter(obj['oid_x'],obj['oid_y'],s=80,c='red',marker='x',label='peak', edgecolors=None)
        if doPlot:
            plt.show()

        if doPrint: 
            print(f"Object detected after filtering: {len(objlist)}")
        if not objlist:
            print('peak has not been properly detected')
            obj ={"px": np.nan,
                  "py": np.nan,
                  "oid_x": np.nan,
                  "oid_y": np.nan,
                  "EE20" : np.nan
                 }
            dict(obj)
        else:
            maxi = np.nanargmax([imeas.getEE(image=data, cx=peak['oid_x'], cy=peak['oid_y'], ee_size=40, roi_size=300)[0] for peak in objlist])
            cx = objlist[maxi]['oid_x']
            cy = objlist[maxi]['oid_y']

        peak = imeas.getPeakData(data, cx, cy, EE=[40], roi_size=roi_size, doPlot=False, com=com, fwhm_radius=fwhm_radius, fwhm_method=fwhm_method)
        peak['experimentId'] = experimentId
        try:
            fca_x = hdr['FCA_X']
        except KeyError:
            fca_x = guessedPos[i]

        peak['fca_x'] = fca_x
        res.append(peak)
        if doPlot:
            plt.show()
        if doPrint:
            print("\n")
        df = pd.DataFrame(res)
    dfs.append(df)

cube = pd.concat(dfs)

   

0
/data/ait/sac/2021-03-01/SAC014257.fits
1
/data/ait/sac/2021-03-01/SAC014258.fits
2
/data/ait/sac/2021-03-01/SAC014259.fits
3
/data/ait/sac/2021-03-01/SAC014260.fits
4
/data/ait/sac/2021-03-01/SAC014261.fits
5
/data/ait/sac/2021-03-01/SAC014262.fits
6
/data/ait/sac/2021-03-01/SAC014263.fits
7
/data/ait/sac/2021-03-01/SAC014264.fits
8
/data/ait/sac/2021-03-01/SAC014265.fits
9
/data/ait/sac/2021-03-01/SAC014266.fits
10
/data/ait/sac/2021-03-01/SAC014267.fits


/software/drp/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/scipy/optimize/minpack.py:447: RuntimeWarning: Number of calls to function has reached maxfev = 800.
  warnings.warn(errors[info][0], RuntimeWarning)


11
/data/ait/sac/2021-03-01/SAC014268.fits
12
/data/ait/sac/2021-03-01/SAC014269.fits
13
/data/ait/sac/2021-03-01/SAC014270.fits
14
/data/ait/sac/2021-03-01/SAC014271.fits
15
/data/ait/sac/2021-03-01/SAC014272.fits
16
/data/ait/sac/2021-03-01/SAC014273.fits
17
/data/ait/sac/2021-03-01/SAC014274.fits
18
/data/ait/sac/2021-03-01/SAC014275.fits
19
/data/ait/sac/2021-03-01/SAC014276.fits
20
/data/ait/sac/2021-03-01/SAC014277.fits
21
/data/ait/sac/2021-03-01/SAC014278.fits
22
/data/ait/sac/2021-03-01/SAC014279.fits
23
/data/ait/sac/2021-03-01/SAC014280.fits
24
/data/ait/sac/2021-03-01/SAC014281.fits
25
/data/ait/sac/2021-03-01/SAC014282.fits
26
/data/ait/sac/2021-03-01/SAC014283.fits
27
/data/ait/sac/2021-03-01/SAC014284.fits
28
/data/ait/sac/2021-03-01/SAC014285.fits
29
/data/ait/sac/2021-03-01/SAC014286.fits
30
/data/ait/sac/2021-03-01/SAC014287.fits
31
/data/ait/sac/2021-03-01/SAC014288.fits
32
/data/ait/sac/2021-03-01/SAC014289.fits
33
/data/ait/sac/2021-03-01/SAC014290.fits
34
/data/ai

/home/rlhoussaine/devel/lam_sps_analysis/python/pfs/lam/imageAnalysis.py:148: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(ee)/np.sum(roi), np.sum(roi)


11
/data/ait/sac/2021-03-01/SAC014388.fits
12
/data/ait/sac/2021-03-01/SAC014389.fits
13
/data/ait/sac/2021-03-01/SAC014390.fits
14
/data/ait/sac/2021-03-01/SAC014391.fits
15
/data/ait/sac/2021-03-01/SAC014392.fits
16
/data/ait/sac/2021-03-01/SAC014393.fits
17
/data/ait/sac/2021-03-01/SAC014394.fits
18
/data/ait/sac/2021-03-01/SAC014395.fits
19
/data/ait/sac/2021-03-01/SAC014396.fits
20
/data/ait/sac/2021-03-01/SAC014397.fits
21
/data/ait/sac/2021-03-01/SAC014398.fits
22
/data/ait/sac/2021-03-01/SAC014399.fits
23
/data/ait/sac/2021-03-01/SAC014400.fits
24
/data/ait/sac/2021-03-01/SAC014401.fits
25
/data/ait/sac/2021-03-01/SAC014402.fits
26
/data/ait/sac/2021-03-01/SAC014403.fits
27
/data/ait/sac/2021-03-01/SAC014404.fits
28
/data/ait/sac/2021-03-01/SAC014405.fits
29
/data/ait/sac/2021-03-01/SAC014406.fits
30
/data/ait/sac/2021-03-01/SAC014407.fits
31
/data/ait/sac/2021-03-01/SAC014408.fits
32
/data/ait/sac/2021-03-01/SAC014409.fits
33
/data/ait/sac/2021-03-01/SAC014410.fits
34
/data/ai

In [ ]:
objlist

In [ ]:
imeas.getEE(image=data, cx=175.87339280732849, cy=33.473473395906886, ee_size=40, roi_size=300)

In [ ]:
cube.plot(x='fca_x',y='EE40',style='*')

In [ ]:
cube.groupby(["experimentId"]).plot(x='fca_x',y='EE40',style='*')

In [ ]:
thres_neighbor = -0.1
doNeighborFilter = True

In [ ]:

if doNeighborFilter :
    tot = []
    for  group_name, series in cube.groupby(["experimentId"]):
        series["EE40_diff"] = series.EE40 - (series.EE40.shift(-1) + series.EE40.shift(1))/2 

        series["EE40_diff"].fillna(0, inplace=True)
        series["EE40_diff_flag"] = (series.EE40_diff> thres_neighbor)
        tot.append(series)
        cube2 = pd.concat(tot)


In [ ]:
cube2[cube2.EE40_diff_flag].groupby(["experimentId"]).plot(x='fca_x',y='EE40',style='*')

In [ ]:
def fitFocusData(cube, corrector=False, doPlot=False, index='fca_x'):
    thfoc_data = []
    
    for experimentId, series in cube.groupby('experimentId'):
        series = series.dropna()
        thfoc = getFocus(series, 'EE40', index, corrector=corrector)
#        for criteria in ['brightness', 'fwhm']:
#            thfoc[criteria] = getFocus(series, criteria, index, corrector=corrector)[criteria]

        thfoc['px'] = np.interp(thfoc[index], series[index], series['px'])
        thfoc['py'] = np.interp(thfoc[index], series[index], series['py'])
        thfoc['experimentId'] = experimentId
        thfoc_data.append(thfoc)
        
    thfoc_data = pd.concat(thfoc_data)

    if doPlot:
        kwargs = dict(grid=True, figsize=(14,10), legend=True, subplots=True)
#        criterias = ['EE20', 'brightness', 'fwhm']
        criterias = ['EE40']
        
        for experimentId, fit in thfoc_data.groupby('experimentId'):
            raw = cube.query("experimentId==%d"%(experimentId))
            axes = fit.set_index(index)[criterias].plot(**kwargs)
            for i, criteria in enumerate(criterias):
                axes[i].plot(raw[index].values, raw[criteria].values, 'o')
                
    return thfoc_data

In [ ]:
thfocModel= fitFocusData(cube2[cube2.EE40_diff_flag], corrector=False, doPlot=True)

In [ ]:
def getFocusModel(fitdata, index='fca_x'):
    data = []
    for experimentId, series in fitdata.groupby('experimentId'):
        series = series.dropna()
#        for criteria in ['EE20', 'brightness', 'fwhm']:
        for criteria in ['EE40']:

            ixmax = series[criteria].idxmax() if criteria !='fwhm' else series[criteria].idxmin()
            focus = series[index][ixmax]
            px = series.px[ixmax]
            py = series.py[ixmax]
            mat = [experimentId, criteria, px, py, focus]
            data.append(tuple(mat))
    
    return pd.DataFrame(data, columns=['experimentId', 'criteria', 'px', 'py', index])

In [ ]:
focusModel = getFocusModel(thfocModel)

In [ ]:
vline = True
plotModel = True
criteria = 'EE40'
index = 'fca_x'

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(111)
j=2

if plotModel:
    ax1.plot(zemaxData.Slitdefocus, zemaxData.MidFiber, 'o', color=colors[0], label='Zemax_MidFiber = %.3f' % zemaxMidFit.focus)
    ax1.plot(zemaxMidFit.x, zemaxMidFit.y, '--', color=colors[0])
    if vline:
        ax1.vlines(color=colors[0], **zemaxMidFit.vline)

    ax1.plot(zemaxData.Slitdefocus, zemaxData.ExtremeFiber, 'o', color=colors[1], label='Zemax_EndFiber = %.3f' % zemaxEndFit.focus)
    ax1.plot(zemaxEndFit.x, zemaxEndFit.y, '--', color=colors[1])
    if vline:
        ax1.vlines(color=colors[1], **zemaxEndFit.vline)

for experimentId, df in cube.groupby('experimentId'):
    fit = thfocModel.query("experimentId==%d"%(experimentId))
    focus = focusModel.query("experimentId==%d and criteria=='%s'"%(experimentId, criteria))
    
    ax1.plot(df[index], df[criteria], 'o', color=colors[j], 
             label='expId%d:%s = %.3f' % (experimentId, experiments.fiber[experimentId], focus[index]))
    ax1.plot(fit[index], fit[criteria], '--', color=colors[j])
    
    if vline:
        ax1.vlines(x=focus[index], ymin=fit[criteria].min(), ymax = fit[criteria].max(), color=colors[j])
    j+=1

    
lns = [line for i, line in enumerate(ax1.get_lines()) if not i % 2]
labs = [line.get_label() for line in lns]

ax1.legend(lns, labs)
ax1.set_xlabel('%s(mm)'%index)
ax1.set_ylabel(criteria)

plt.title('Slit Through focus : Zemax vs Engineering_Fibers \n Criteria : %s doBck=True' %criteria)
plt.grid()

In [ ]:
distance_fibres = 138.84
delta_focus = (1.086 - 0.633)
angle=np.arctan(delta_focus/distance_fibres)
angle_degre=(angle*180)/(np.pi)
print(angle_degre)


In [ ]:
((0.648+0.658)/2) - 0.309


In [ ]:
0.270-0.0062

In [ ]:
1.532-0.885